# Selector Group Chat

SelectorGroupChat is a group chat similar to RoundRobinGroupChat, but with a model-based next speaker selection mechanism.

In [1]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

# Load API key
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Model client
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [2]:
planning_agent = AssistantAgent(
    name = 'PlanningAgent',
    description= 'An agent for planning tasks,this agent should be the first to engage when given a new task.',
    model_client=model_client,
    system_message='''
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are :
        WebSearchAgent : Searches for information.
        DataAnalystAgent : Performs calculations

    You only plan and delegate tasks - you do not exectue them yourself.

    When assigning tasks, use the below format:
    1. <agent> : <task>

    After all the tasks are completed, summarize the findings and end with "TERMINATE"
    ''',
)

In [3]:
def search_web_tool(query:str)-> str:
    # Simulate a web search
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."

web_search_agent = AssistantAgent(
    name = 'WebSearchAgent',
    description= 'An agent for searching the web for information.',
    model_client=model_client,
    tools = [search_web_tool],
    system_message='''
        You are a web search agent.
        Your only tool is search_web_tool - use it to find the information you need.

        You make only one search call at a time.
        
        Once you have the results, you never do calculations or data analysis on them.
    ''',
)

In [4]:
def percentage_change_tool(start:float, end:float) -> float:
    # Calculate percentage change
    if start == 0:
        return 0
    return ((end - start) / start) * 100


data_analyst_agent = AssistantAgent(
    name = 'DataAnalystAgent',
    description= 'An agent for performing calculations and data analysis.',
    model_client=model_client,
    tools= [percentage_change_tool],
    system_message='''
        You are a data analyst agent.
        Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.

        If you have not seen the data, ask for it.

    ''',
)

# Termination Conditions

In [5]:
from autogen_agentchat.conditions import MaxMessageTermination,TextMentionTermination

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=20)
combined_termination = text_mention_termination | max_messages_termination

In [6]:
selector_prompt = '''
Select an agent to perform the task.

{roles}

Current conversation history :
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planning agent has assigned task before other agents start working.
Only Select one agent.
'''

In [7]:
'''
Try not to overload the selector prompt with too much information.


selector_prompt (str, optional) – The prompt template to use for selecting the next speaker. 

Available fields: ‘{roles}’, ‘{participants}’, and ‘{history}’. 
1. {participants} is the names of candidates for selection. The format is [“<name1>”, “<name2>”, …]. 
2. {roles} is a newline-separated list of names and descriptions of the candidate agents. The format for each line is: “<name> : <description>”. 

3. {history} is the conversation history formatted as a double newline separated of names and message content. The format for each message is: “<name> : <message content>”.
'''

'\nTry not to overload the selector prompt with too much information.\n\n\nselector_prompt (str, optional) – The prompt template to use for selecting the next speaker. \n\nAvailable fields: ‘{roles}’, ‘{participants}’, and ‘{history}’. \n1. {participants} is the names of candidates for selection. The format is [“<name1>”, “<name2>”, …]. \n2. {roles} is a newline-separated list of names and descriptions of the candidate agents. The format for each line is: “<name> : <description>”. \n\n3. {history} is the conversation history formatted as a double newline separated of names and message content. The format for each message is: “<name> : <message content>”.\n'

In [8]:
selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True)

In [9]:
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

In [10]:
from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To address this inquiry, we'll follow these steps:

1. Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.
2. Locate the player's total rebounds for both the 2007-2008 and 2008-2009 NBA seasons.
3. Calculate the percentage change in the player's total rebounds between these two seasons.

Let's assign the tasks:

1. WebSearchAgent: Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.
2. WebSearchAgent: Find the total rebounds for this player in the 2007-2008 NBA season.
3. WebSearchAgent: Find the total rebounds for this player in the 2008-2009 NBA season.
4. DataAnalystAgent: Calculate the percentage change in the player's total rebounds between the 2007-200

TaskResult(messages=[TextMessage(id='84c155bd-4c70-4b7e-8a51-b03a1ec32779', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 2, 9, 44, 54, 470464, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='79705cf5-f1a7-4a63-b429-00f978940be0', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=165, completion_tokens=186), metadata={}, created_at=datetime.datetime(2025, 8, 2, 9, 45, 1, 162215, tzinfo=datetime.timezone.utc), content="To address this inquiry, we'll follow these steps:\n\n1. Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.\n2. Locate the player's total rebounds for both the 2007-2008 and 2008-2009 NBA seasons.\n3. Calculate the percentage change in the player's total rebounds between these two

# Custom Selector Function

In [16]:
s = "Suraj"
s[-1]

'j'

In [17]:
planning_agent.name

'PlanningAgent'

In [13]:
from typing import Sequence
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage

def selector_func(messages : Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if(messages[-1].source != planning_agent.name):
        return planning_agent.name
    return None

In [14]:
await selector_team.reset()
selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=selector_func)

In [15]:
from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To find out who was the Miami Heat player with the highest points in the 2006-2007 season and the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons, we need to follow these steps:

1. Identify the Miami Heat player with the highest points in the 2006-2007 season.
2. Collect the data on his total rebounds for both the 2007-2008 and 2008-2009 seasons.
3. Calculate the percentage change in his total rebounds between these two seasons.

Here are the tasked steps:

1. WebSearchAgent: Identify the Miami Heat player with the highest points scored in the 2006-2007 NBA season.
2. WebSearchAgent: Find the total rebounds for that player in the 2007-2008 NBA season.
3. WebSearchAgent: Find the tota

TaskResult(messages=[TextMessage(id='f6a1573b-c77e-4765-8aea-61c1d05715e7', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 2, 9, 45, 15, 533457, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='c8a667e0-a0f8-4c2d-9ef7-a758e0456620', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=165, completion_tokens=233), metadata={}, created_at=datetime.datetime(2025, 8, 2, 9, 45, 20, 817626, tzinfo=datetime.timezone.utc), content="To find out who was the Miami Heat player with the highest points in the 2006-2007 season and the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons, we need to follow these steps:\n\n1. Identify the Miami Heat player with the highest points in the 2006-2007 season.\n2. Collect t